In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pylab import rcParams
import warnings
warnings.filterwarnings('ignore')
rcParams['figure.figsize'] = 8,4.5


In [ ]:
df1 = pd.read_csv('prices.csv',header =0)
df1

In [ ]:
print(df1.isnull().sum())

In [ ]:
df1.dtypes

In [ ]:
x = df1.iloc[:,2:6].values
x = np.delete(x,1,axis = 1)
x

In [ ]:
y = df1.iloc[:,3].values
y

In [ ]:
display(pd.DataFrame(x).corr())

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x= StandardScaler()
x = sc_x.fit_transform(x)
print (x)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1)
PCA_X = pca.fit_transform(x)
print (pd.DataFrame(PCA_X))


In [ ]:
print (pca.explained_variance_ratio_)

In [ ]:

from sklearn.model_selection import train_test_split
PCA_X_train,PCA_X_test, y_train,y_test = train_test_split(PCA_X,y,test_size=0.2,random_state=42)
print (PCA_X.shape)
print (PCA_X_train.shape)
print (PCA_X_test.shape)



In [ ]:
#to creat linear model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(PCA_X_train,y_train)


In [ ]:
y_pred = lr.predict(PCA_X_test)
y_pred

In [ ]:
#to check accuracy
from sklearn.metrics import r2_score
acc = r2_score(y_test,y_pred)
acc

In [ ]:
def Plotting_Prediction_VsActual(trainY,train_pred):
    trainY = pd.DataFrame(trainY).tail(100)
    train_pred = pd.DataFrame(train_pred).tail(100)
    print("Red - Predicted Stock Prices  ,  Blue - Actual Stock Prices")
    plt.rcParams["figure.figsize"] = (15,7)
    plt.plot(trainY  ,color='blue', label='Actual', alpha=0.9)
    plt.plot(train_pred, color='red', label='Prediction', alpha=0.5)
    plt.xlabel('Time')
    plt.ylabel('Stock Prices')
    plt.legend(fontsize = 14)
    plt.title('Pediction VS Reality Over Time')
    plt.grid(True)
    plt.show()


In [ ]:
def Model_Details(y_test, y_pred):
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    print(f"R² Score        : {r2:.4f}")
    print(f"MAE (Error)     : {mae:.2f}")
    print(f"RMSE (Error)    : {rmse:.2f}")
    print(f"Maximum Actual Price: {max(y_test):.2f}")
    print(f"Maximum Predicted Price  : {max(y_pred):.2f}")
    print(f"Minimum Actual Price   : {min(y_test):.2f}")
    print(f"Minimum Predicted Price  : {min(y_pred):.2f}")


In [ ]:
def Model_Prediction_Actual_Graph(x,y):
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    sc_x= StandardScaler()
    x = sc_x.fit_transform(x)
    pca = PCA(n_components = 1)
    PCA_X = pca.fit_transform(x)
    PCA_X_train,PCA_X_test, y_train,y_test = train_test_split(PCA_X,y,test_size=0.2,random_state=42)
    lr = LinearRegression()
    lr.fit(PCA_X_train,y_train)
    y_pred = lr.predict(PCA_X_test)
    Plotting_Prediction_VsActual(y_test,y_pred)
    Model_Details(y_test,y_pred)

In [ ]:
def x_y_values(data):
    x = data.iloc[:,2:6].values
    x = np.delete(x,1,axis = 1)
    y = data.iloc[:,3].values
    noise = np.random.normal(0, y.std() * 0.2, size=len(y))  
    y = y + noise
    return x,y

In [ ]:
def print_sideways(data_list, cols=4):
    from math import ceil
    rows = ceil(len(data_list) / cols)
    for r in range(rows):
        row_items = []
        for c in range(cols):
            idx = r + c * rows
            if idx < len(data_list):
                row_items.append(f"{data_list[idx]:<20}")
        print(" | ".join(row_items))

In [ ]:
def Model_Prediction():
    df1 = pd.read_csv('prices.csv',header =0 )
    df2 = pd.read_csv('securities.csv',header = 0)
    codes = df2['Ticker symbol'].unique()
    w ='''Select Feature to Use:
    1.Prediction Vs Actual Stock Prices for One Company
    2.Prediction Vs Actual Stock Prices for Multiple Companies
    3.Prediction Vs Actual Stock Prices for All Companies
    '''
    print(w)
    ch_1 = int(input("Enter Choice(1/2/3):"))
    if(ch_1 == 1):
        print_sideways(codes,6)
        company_required = input("Enter Company Code to Search:")
        f = df2.loc[df2['Ticker symbol'] == company_required].values[0]
        col = df2.columns
        for j in range(len(col)):
            if(col[j]!='Date first added'):
                print(col[j],':',f[j])
        data = df1.loc[df1.symbol.str.startswith(company_required) , :]
        x,y = x_y_values(data)
        Model_Prediction_Actual_Graph(x,y)
    elif(ch_1 == 2):
        print_sideways(codes,6)
        custom_codes = eval(input("Enter A List of Companies Using their Codes:"))
        for i in custom_codes:
            f = df2.loc[df2['Ticker symbol'] == i].values[0]
            col = df2.columns
            for j in range(len(col)):
                if(col[j]!='Date first added'):
                    print(col[j],':',f[j])
            data = df1.loc[df1.symbol.str.startswith(i) , :]
            x,y = x_y_values(data)
            Model_Prediction_Actual_Graph(x,y)
    elif(ch_1==3):
        print_sideways(codes,6)
        for i in codes:
            f = df2.loc[df2['Ticker symbol'] == i].values[0]
            col = df2.columns
            for j in range(len(col)):
                if(col[j]!='Date first added'):
                    print(col[j],':',f[j])
            data = df1.loc[df1.symbol.str.startswith(i) , :]
            x,y = x_y_values(data)
            Model_Prediction_Actual_Graph(x,y)
    else:
        print("Enter A Valid Choice")
        
            


In [ ]:
Model_Prediction()